In [7]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from score_drives import get_drive_stats

In [12]:
df = get_drive_stats(2009, 2019, exclude_playoffs=False,
                     exclude_blowouts=False, opponent_adjustment=False,
                     data_path='/home/jovyan/work/data')

2010122604.json '2010122604'
min() arg is an empty sequence dict_keys([])


In [13]:
df.head()

,away_final_score,away_team,defensive_team,drive_time,end_quarter,end_time,first_play_desc,game_id,home_final_score,home_score_diff_last_quarter,...,defensive_win,tie,1_quarter_time,2_quarter_time,3_quarter_time,4_quarter_time,5_quarter_time,offense_home,defense_home,nfl_avg_score
1552,10,TEN,TEN,1.733333,1,13.266667,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,2009091000,13,0,...,0,0,1.733333,0.0,0.0,0.0,0.0,True,False,2.245383
1553,10,TEN,PIT,1.866667,1,11.400000,(13:16) C.Johnson up the middle to TEN 2 for n...,2009091000,13,0,...,1,0,1.866667,0.0,0.0,0.0,0.0,False,True,0.963020
1554,10,TEN,TEN,3.066667,1,8.333333,(11:24) B.Roethlisberger pass short right to M...,2009091000,13,0,...,0,0,3.066667,0.0,0.0,0.0,0.0,True,False,2.460535
1555,10,TEN,PIT,1.600000,1,6.733333,(8:20) C.Johnson right end to TEN 43 for 32 ya...,2009091000,13,0,...,1,0,1.600000,0.0,0.0,0.0,0.0,False,True,1.439739
1556,10,TEN,TEN,1.916667,1,4.816667,(6:44) R.Mendenhall up the middle to PIT 28 fo...,2009091000,13,0,...,0,0,1.916667,0.0,0.0,0.0,0.0,True,False,1.546391


In [14]:
df.columns

Index(['away_final_score', 'away_team', 'defensive_team', 'drive_time',
       'end_quarter', 'end_time', 'first_play_desc', 'game_id',
       'home_final_score', 'home_score_diff_last_quarter', 'home_team',
       'last_play_desc', 'n_plays', 'offensive_team', 'penalty_yards',
       'result', 'start_quarter', 'start_time', 'start_yard_line',
       'yards_gained', 'drive_id', 'season', 'unique_game_flag',
       'game_in_season', 'is_playoffs', 'total_yards', 'end_yard_line',
       'next_start_yard_line', 'next_end_yard_line', 'next_offensive_team',
       'start_yard_line_bin', 'end_yard_line_bin', 'next_start_yard_line_bin',
       'expected_points', 'offensive_points', 'is_touchdown', 'is_field_goal',
       'is_score', 'is_interception', 'is_fumble', 'dst_points', 'home_points',
       'away_points', 'offensive_team_score_end', 'defensive_team_score_end',
       'offensive_team_score_start', 'defensive_team_score_start',
       'home_score_end', 'away_score_end', 'home_score_sta

In [15]:
df['game_id'].values

array(['2009091000', '2009091000', '2009091000', ..., '2019090901',
       '2019090901', '2019090901'], dtype=object)

In [16]:
df.loc[df['game_id'] >= '2010090100', 'game_id'].values

array(['2010090900', '2010090900', '2010090900', ..., '2019090901',
       '2019090901', '2019090901'], dtype=object)

In [ ]:
train_start = '2009091000'
test_start = '2010090900'